In [1]:
import pandas as pd 
import pickle
from scipy.stats import poisson

In [2]:
dict_table = pickle.load(open('dict_tablas', 'rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixtures.csv')

<h1> 1 Calcular Team Strenght </h1>

In [3]:
# Dividir df en df_home y df_away 
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [4]:
#Renombar columnas
df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

In [5]:
#Concatenar df_home y df_away , hacer group por team y calcular promedio
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


<h2> Predict Points </h2>

In [6]:
def predict_points(home, away):
    #Check if teams are in the index
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [7]:
predict_points('Argentina','Mexico')

(2.3129151525530505, 0.5378377125059863)

<h2> 3. Prediciendo el mundial </h2>

In [8]:
# Dividiendo fixture en group , octavos, cuartos.
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [9]:
# Correr todos los partidos de la fase de grupo y actualizar las tablas de cada grupo
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [11]:
dict_table['Grupo A']

,Team,Pts
0,Mexico,8.0
1,Croatia,8.0
2,Brazil (H),7.0
3,Cameroon,1.0


## Knock out


In [12]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [13]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Winners Group A,Match 49,Runners-up Group B,2022,?
49,Winners Group C,Match 50,Runners-up Group D,2022,?
50,Winners Group D,Match 52,Runners-up Group C,2022,?
51,Winners Group B,Match 51,Runners-up Group A,2022,?
52,Winners Group E,Match 53,Runners-up Group F,2022,?
53,Winners Group G,Match 54,Runners-up Group H,2022,?
54,Winners Group F,Match 55,Runners-up Group E,2022,?
55,Winners Group H,Match 56,Runners-up Group G,2022,?


In [14]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [15]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Winners Group A,Match 49,Runners-up Group B,2022,Runners-up Group B
49,Winners Group C,Match 50,Runners-up Group D,2022,Runners-up Group D
50,Winners Group D,Match 52,Runners-up Group C,2022,Runners-up Group C
51,Winners Group B,Match 51,Runners-up Group A,2022,Runners-up Group A
52,Winners Group E,Match 53,Runners-up Group F,2022,Runners-up Group F
53,Winners Group G,Match 54,Runners-up Group H,2022,Runners-up Group H
54,Winners Group F,Match 55,Runners-up Group E,2022,Runners-up Group E
55,Winners Group H,Match 56,Runners-up Group G,2022,Runners-up Group G


## Quarter Final

In [16]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [17]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Runners-up Group F,Match 58,Runners-up Group H,2022,?
57,Runners-up Group B,Match 57,Runners-up Group D,2022,?
58,Runners-up Group E,Match 60,Runners-up Group G,2022,?
59,Runners-up Group A,Match 59,Runners-up Group C,2022,?


In [18]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Runners-up Group F,Match 58,Runners-up Group H,2022,Runners-up Group H
57,Runners-up Group B,Match 57,Runners-up Group D,2022,Runners-up Group D
58,Runners-up Group E,Match 60,Runners-up Group G,2022,Runners-up Group G
59,Runners-up Group A,Match 59,Runners-up Group C,2022,Runners-up Group C


## Semifinal


In [19]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Runners-up Group D,Match 61,Runners-up Group H,2022,?
61,Runners-up Group C,Match 62,Runners-up Group G,2022,?


In [20]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Runners-up Group D,Match 61,Runners-up Group H,2022,Runners-up Group H
61,Runners-up Group C,Match 62,Runners-up Group G,2022,Runners-up Group G


## Final

In [22]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Runners-up Group H,Match 64,Runners-up Group G,2022,?


In [23]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Runners-up Group H,Match 64,Runners-up Group G,2022,Runners-up Group G
